# PANDA-PLUS-Bench: Evaluating WSI-Specific Feature Collapse

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dellacortelab/PANDA-PLUS-Bench/blob/main/PANDA_PLUS_Bench_Evaluation.ipynb)

This notebook enables evaluation of pathology foundation models on the PANDA-PLUS-Bench benchmark for measuring WSI-specific feature collapse.

**What this notebook does:**
1. Loads pre-augmented patches from the benchmark dataset
2. Extracts embeddings using your foundation model (or default Phikon)
3. Computes robustness metrics (within-slide vs cross-slide accuracy, etc.)
4. Compares your model against published results from 7 foundation models
5. Generates publication-quality visualizations

**Citation:**
```
Ebbert J, Della Corte D. PANDA-PLUS-Bench: A Benchmark for Evaluating 
WSI-Specific Feature Collapse in Pathology Foundation Models. 2025.
```

---

## Step 0: Setup and Installation

Run this cell first to install required packages (~2-3 minutes).

In [ ]:
# Install required packages
!pip install -q datasets huggingface_hub transformers timm torch torchvision
!pip install -q scikit-learn scipy matplotlib seaborn tqdm pandas numpy

import sys
import warnings
warnings.filterwarnings('ignore')

# Verify GPU availability
import torch
if torch.cuda.is_available():
    print(f"✓ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"  Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    DEVICE = torch.device('cuda')
else:
    print("⚠ No GPU detected. Embedding extraction will be slow.")
    print("  Go to Runtime → Change runtime type → GPU")
    DEVICE = torch.device('cpu')

print("\n✓ Setup complete!")

## Step 1: Configuration

**Configure your evaluation here:**
- Set your HuggingFace token (if using gated models like UNI, Virchow)
- Choose which foundation model to evaluate
- Select augmentation conditions to test

In [ ]:
#@title Configuration {display-mode: "form"}

#@markdown ### HuggingFace Token (optional)
#@markdown Required only for gated models (UNI, Virchow, etc.)
#@markdown Get your token at: https://huggingface.co/settings/tokens
HF_TOKEN = ""  #@param {type:"string"}

#@markdown ### Model Selection
#@markdown Choose a model to evaluate, or enter a custom HuggingFace model ID
MODEL_CHOICE = "owkin/phikon"  #@param ["owkin/phikon", "owkin/phikon-v2", "MahmoodLab/UNI", "paige-ai/Virchow", "paige-ai/Virchow2", "custom"]
CUSTOM_MODEL_ID = ""  #@param {type:"string"}

#@markdown ### Augmentation Conditions
#@markdown Select which augmentation conditions to evaluate
EVAL_BASELINE = True  #@param {type:"boolean"}
EVAL_COLOR_JITTER = False  #@param {type:"boolean"}
EVAL_GRAYSCALE = False  #@param {type:"boolean"}
EVAL_GAUSSIAN_NOISE = False  #@param {type:"boolean"}
EVAL_HEAVY_GEOMETRIC = False  #@param {type:"boolean"}
EVAL_COMBINED_AGGRESSIVE = True  #@param {type:"boolean"}
EVAL_MACENKO = False  #@param {type:"boolean"}
EVAL_HED = False  #@param {type:"boolean"}

#@markdown ### Evaluation Settings
N_NEIGHBORS = 5  #@param {type:"integer"}
BATCH_SIZE = 32  #@param {type:"integer"}

# Process configuration
MODEL_ID = CUSTOM_MODEL_ID if MODEL_CHOICE == "custom" else MODEL_CHOICE

AUGMENTATIONS_TO_EVAL = []
if EVAL_BASELINE: AUGMENTATIONS_TO_EVAL.append("baseline")
if EVAL_COLOR_JITTER: AUGMENTATIONS_TO_EVAL.append("color_jitter")
if EVAL_GRAYSCALE: AUGMENTATIONS_TO_EVAL.append("grayscale")
if EVAL_GAUSSIAN_NOISE: AUGMENTATIONS_TO_EVAL.append("gaussian_noise")
if EVAL_HEAVY_GEOMETRIC: AUGMENTATIONS_TO_EVAL.append("heavy_geometric")
if EVAL_COMBINED_AGGRESSIVE: AUGMENTATIONS_TO_EVAL.append("combined_aggressive")
if EVAL_MACENKO: AUGMENTATIONS_TO_EVAL.append("macenko_normalization")
if EVAL_HED: AUGMENTATIONS_TO_EVAL.append("hed_stain_augmentation")

print("Configuration Summary:")
print(f"  Model: {MODEL_ID}")
print(f"  Augmentations: {AUGMENTATIONS_TO_EVAL}")
print(f"  HF Token: {'Set' if HF_TOKEN else 'Not set (using public models only)'}")
print(f"  Device: {DEVICE}")

## Step 2: Load Benchmark Dataset

Downloads patches from HuggingFace Hub (~2-5 GB depending on augmentations selected).

In [ ]:
from datasets import load_dataset
import json
import numpy as np

# Dataset repository
DATASET_REPO = "dellacorte/PANDA-PLUS-Bench"

print("Loading PANDA-PLUS-Bench dataset...")
print("(First run will download ~2-5 GB, subsequent runs use cache)\n")

# Load the dataset
try:
    dataset = load_dataset(DATASET_REPO, token=HF_TOKEN if HF_TOKEN else None)
    print(f"✓ Dataset loaded successfully!")
    print(f"  Available splits: {list(dataset.keys())}")
    
    # Show dataset info
    sample_split = list(dataset.keys())[0]
    print(f"  Total patches in '{sample_split}': {len(dataset[sample_split])}")
    print(f"  Features: {dataset[sample_split].features}")
    
except Exception as e:
    print(f"✗ Error loading dataset: {e}")
    print("\nTroubleshooting:")
    print("  1. Check that the dataset exists at the specified repo")
    print("  2. If private, ensure HF_TOKEN is set correctly")
    raise

# Load paper results for comparison
print("\nLoading published benchmark results...")
try:
    # Load from GitHub repo
    import urllib.request
    paper_results_url = "https://raw.githubusercontent.com/dellacortelab/PANDA-PLUS-Bench/main/paper_results.json"
    with urllib.request.urlopen(paper_results_url) as response:
        PAPER_RESULTS = json.loads(response.read().decode())
    print(f"✓ Loaded results for {len(PAPER_RESULTS)} models from paper")
    print(f"  Models: {list(PAPER_RESULTS.keys())}")
except Exception as e:
    print(f"⚠ Paper results not found: {e}")
    print("  Comparison will be skipped.")
    PAPER_RESULTS = {}

## Step 3: Load Foundation Model

Loads the selected foundation model for embedding extraction.

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoImageProcessor
import timm
from torchvision import transforms

def load_foundation_model(model_id, hf_token=None):
    """
    Load a pathology foundation model.
    
    Supports:
    - Phikon / Phikon-v2 (owkin)
    - UNI / UNI2 (MahmoodLab)
    - Virchow / Virchow2 (paige-ai)
    - Any timm-compatible model
    - Any HuggingFace transformers model
    """
    print(f"Loading model: {model_id}")
    
    model = None
    processor = None
    embed_dim = None
    
    # Try HuggingFace transformers first
    try:
        model = AutoModel.from_pretrained(
            model_id, 
            token=hf_token,
            trust_remote_code=True
        )
        
        try:
            processor = AutoImageProcessor.from_pretrained(
                model_id, 
                token=hf_token
            )
        except:
            processor = None
        
        # Get embedding dimension
        if hasattr(model.config, 'hidden_size'):
            embed_dim = model.config.hidden_size
        elif hasattr(model, 'embed_dim'):
            embed_dim = model.embed_dim
        else:
            embed_dim = None
            
        print(f"  ✓ Loaded via HuggingFace transformers")
        
    except Exception as e:
        print(f"  HuggingFace loading failed: {e}")
        print(f"  Trying timm...")
        
        # Try timm
        try:
            model = timm.create_model(model_id, pretrained=True, num_classes=0)
            embed_dim = model.num_features
            processor = None
            print(f"  ✓ Loaded via timm")
        except Exception as e2:
            raise ValueError(f"Could not load model '{model_id}' via HuggingFace or timm: {e2}")
    
    model = model.to(DEVICE)
    model.eval()
    
    # Default transform if no processor
    if processor is None:
        default_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])
    else:
        default_transform = None
    
    print(f"  Embedding dimension: {embed_dim}")
    print(f"  Device: {DEVICE}")
    
    return model, processor, default_transform, embed_dim


# Load the model
model, processor, transform, embed_dim = load_foundation_model(MODEL_ID, HF_TOKEN)
print("\n✓ Model ready for embedding extraction!")

## Step 4: Extract Embeddings

Extracts embeddings for all patches in selected augmentation conditions.

**Expected time:** ~5-15 minutes per augmentation on Colab GPU (T4)

In [ ]:
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm
from PIL import Image
import numpy as np

class PatchDataset(Dataset):
    """Dataset wrapper for benchmark patches."""
    def __init__(self, hf_dataset, processor=None, transform=None):
        self.dataset = hf_dataset
        self.processor = processor
        self.transform = transform
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        item = self.dataset[idx]
        
        # Get image (handle different formats)
        if 'image' in item:
            image = item['image']
        elif 'patch' in item:
            image = item['patch']
        else:
            raise KeyError(f"No image field found. Available: {item.keys()}")
        
        # Convert to PIL if needed
        if isinstance(image, np.ndarray):
            image = Image.fromarray(image)
        
        # Apply preprocessing
        if self.processor is not None:
            inputs = self.processor(images=image, return_tensors="pt")
            pixel_values = inputs['pixel_values'].squeeze(0)
        elif self.transform is not None:
            pixel_values = self.transform(image)
        else:
            raise ValueError("No processor or transform provided")
        
        return {
            'pixel_values': pixel_values,
            'label': item.get('label', -1),
            'slide_id': item.get('slide_id', 'unknown')
        }


@torch.no_grad()
def extract_embeddings(model, dataloader, device):
    """
    Extract embeddings from all patches.
    
    Returns:
        embeddings: numpy array of shape (n_patches, embed_dim)
        labels: numpy array of shape (n_patches,)
        slide_ids: numpy array of shape (n_patches,)
    """
    model.eval()
    
    all_embeddings = []
    all_labels = []
    all_slide_ids = []
    
    for batch in tqdm(dataloader, desc="Extracting embeddings"):
        pixel_values = batch['pixel_values'].to(device)
        
        # Forward pass
        outputs = model(pixel_values)
        
        # Extract embeddings (handle different output formats)
        if hasattr(outputs, 'last_hidden_state'):
            embeddings = outputs.last_hidden_state[:, 0, :]
        elif hasattr(outputs, 'pooler_output') and outputs.pooler_output is not None:
            embeddings = outputs.pooler_output
        elif isinstance(outputs, torch.Tensor):
            embeddings = outputs
        else:
            raise ValueError(f"Unknown output format: {type(outputs)}")
        
        all_embeddings.append(embeddings.cpu().numpy())
        all_labels.extend(batch['label'].numpy())
        all_slide_ids.extend(batch['slide_id'])
    
    embeddings = np.vstack(all_embeddings)
    labels = np.array(all_labels)
    slide_ids = np.array(all_slide_ids)
    
    return embeddings, labels, slide_ids


# Extract embeddings for each augmentation
print(f"\nExtracting embeddings for {len(AUGMENTATIONS_TO_EVAL)} augmentation(s)...\n")

all_results = {}

for aug in AUGMENTATIONS_TO_EVAL:
    print(f"\n{'='*60}")
    print(f"Processing: {aug}")
    print(f"{'='*60}")
    
    if aug not in dataset:
        print(f"  ⚠ Augmentation '{aug}' not found in dataset. Skipping.")
        continue
    
    patch_dataset = PatchDataset(
        dataset[aug], 
        processor=processor, 
        transform=transform
    )
    
    dataloader = DataLoader(
        patch_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=2,
        pin_memory=True
    )
    
    print(f"  Patches: {len(patch_dataset)}")
    print(f"  Batches: {len(dataloader)}")
    
    embeddings, labels, slide_ids = extract_embeddings(model, dataloader, DEVICE)
    
    all_results[aug] = {
        'embeddings': embeddings,
        'labels': labels,
        'slide_ids': slide_ids
    }
    
    print(f"  ✓ Embeddings shape: {embeddings.shape}")
    print(f"  ✓ Unique slides: {len(np.unique(slide_ids))}")
    print(f"  ✓ Unique labels: {np.unique(labels)}")

print(f"\n\n✓ Embedding extraction complete!")
print(f"  Processed {len(all_results)} augmentation(s)")

## Step 5: Compute Benchmark Metrics

Computes the standard PANDA-PLUS-Bench metrics:
- **Within-slide accuracy**: Classification using patches from same slide
- **Cross-slide accuracy**: Leave-one-slide-out classification
- **Accuracy gap**: Within - Cross (higher = more collapse)
- **Silhouette scores**: Cluster quality by class and by slide
- **Confusion attribution**: Entropy of kNN label distribution

In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import silhouette_score
from scipy.stats import entropy

def compute_within_cross_accuracy(embeddings, labels, slide_ids, n_neighbors=5):
    """Compute within-slide and cross-slide classification accuracy."""
    unique_slides = np.unique(slide_ids)
    within_scores = []
    cross_scores = []
    
    # Within-slide accuracy
    for slide in unique_slides:
        slide_mask = slide_ids == slide
        slide_embeddings = embeddings[slide_mask]
        slide_labels = labels[slide_mask]
        
        if len(slide_embeddings) < n_neighbors + 1:
            continue
        
        n_test = max(1, len(slide_embeddings) // 5)
        indices = np.random.permutation(len(slide_embeddings))
        train_idx, test_idx = indices[:-n_test], indices[-n_test:]
        
        knn = KNeighborsClassifier(n_neighbors=min(n_neighbors, len(train_idx)-1))
        knn.fit(slide_embeddings[train_idx], slide_labels[train_idx])
        within_scores.append(knn.score(slide_embeddings[test_idx], slide_labels[test_idx]))
    
    # Cross-slide accuracy
    logo = LeaveOneGroupOut()
    for train_idx, test_idx in logo.split(embeddings, labels, slide_ids):
        X_train, X_test = embeddings[train_idx], embeddings[test_idx]
        y_train, y_test = labels[train_idx], labels[test_idx]
        
        knn = KNeighborsClassifier(n_neighbors=n_neighbors)
        knn.fit(X_train, y_train)
        cross_scores.append(knn.score(X_test, y_test))
    
    return np.mean(within_scores), np.mean(cross_scores)

# Compute metrics for each augmentation
print("\nComputing metrics...\n")
metrics = {}

for aug, results in all_results.items():
    print(f"Computing metrics for {aug}...")
    
    within_acc, cross_acc = compute_within_cross_accuracy(
        results['embeddings'],
        results['labels'],
        results['slide_ids'],
        n_neighbors=N_NEIGHBORS
    )
    
    metrics[aug] = {
        'within_accuracy': within_acc,
        'cross_accuracy': cross_acc,
        'accuracy_gap': within_acc - cross_acc
    }
    
    print(f"  Within-slide accuracy: {within_acc:.3f}")
    print(f"  Cross-slide accuracy: {cross_acc:.3f}")
    print(f"  Accuracy gap: {within_acc - cross_acc:.3f}\n")

print("✓ Metrics computed!")

## Step 6: Display Results

Display your model's results in a summary table.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Create results dataframe
results_data = []
for aug, m in metrics.items():
    results_data.append({
        'Augmentation': aug,
        'Within-Slide Acc': m['within_accuracy'],
        'Cross-Slide Acc': m['cross_accuracy'],
        'Accuracy Gap': m['accuracy_gap']
    })

results_df = pd.DataFrame(results_data)

# Display table
print(f"\n{'='*70}")
print(f"Results for {MODEL_ID}")
print(f"{'='*70}\n")
display_df = results_df.copy()
display_df['Within-Slide Acc'] = display_df['Within-Slide Acc'].apply(lambda x: f"{x:.3f}")
display_df['Cross-Slide Acc'] = display_df['Cross-Slide Acc'].apply(lambda x: f"{x:.3f}")
display_df['Accuracy Gap'] = display_df['Accuracy Gap'].apply(lambda x: f"{x:.3f}")
print(display_df.to_string(index=False))
print(f"\n{'='*70}")

# Plot results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Within vs Cross-slide accuracy
ax1 = axes[0]
x = range(len(results_df))
width = 0.35
ax1.bar([i - width/2 for i in x], results_df['Within-Slide Acc'], width, 
        label='Within-Slide', alpha=0.8, color='#2ecc71')
ax1.bar([i + width/2 for i in x], results_df['Cross-Slide Acc'], width,
        label='Cross-Slide', alpha=0.8, color='#3498db')
ax1.set_xlabel('Augmentation', fontsize=12)
ax1.set_ylabel('Accuracy', fontsize=12)
ax1.set_title('Within-Slide vs Cross-Slide Accuracy', fontsize=14, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(results_df['Augmentation'], rotation=45, ha='right')
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# Plot 2: Accuracy gap
ax2 = axes[1]
colors = ['#e74c3c' if gap > 0.2 else '#f39c12' if gap > 0.1 else '#2ecc71' 
          for gap in results_df['Accuracy Gap']]
ax2.bar(x, results_df['Accuracy Gap'], alpha=0.8, color=colors)
ax2.axhline(y=0.2, color='red', linestyle='--', linewidth=1, alpha=0.5, label='High collapse (>0.20)')
ax2.axhline(y=0.1, color='orange', linestyle='--', linewidth=1, alpha=0.5, label='Moderate collapse (>0.10)')
ax2.set_xlabel('Augmentation', fontsize=12)
ax2.set_ylabel('Accuracy Gap', fontsize=12)
ax2.set_title('Feature Collapse (Accuracy Gap)', fontsize=14, fontweight='bold')
ax2.set_xticks(x)
ax2.set_xticklabels(results_df['Augmentation'], rotation=45, ha='right')
ax2.legend(loc='upper right')
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Summary statistics
print("\n" + "="*70)
print("Summary Statistics")
print("="*70)
print(f"Average Within-Slide Accuracy: {results_df['Within-Slide Acc'].mean():.3f}")
print(f"Average Cross-Slide Accuracy:  {results_df['Cross-Slide Acc'].mean():.3f}")
print(f"Average Accuracy Gap:          {results_df['Accuracy Gap'].mean():.3f}")
print(f"Max Accuracy Gap:              {results_df['Accuracy Gap'].max():.3f}")
print(f"Min Accuracy Gap:              {results_df['Accuracy Gap'].min():.3f}")

## Step 7: Compare with Published Results

Compare your model's performance against published foundation models.

In [ ]:
if PAPER_RESULTS:
    # Prepare comparison data
    comparison_data = []
    
    # Add your model results
    for aug in AUGMENTATIONS_TO_EVAL:
        if aug in metrics:
            comparison_data.append({
                'Model': MODEL_ID.split('/')[-1],
                'Augmentation': aug,
                'Within-Slide Acc': metrics[aug]['within_accuracy'],
                'Cross-Slide Acc': metrics[aug]['cross_accuracy'],
                'Accuracy Gap': metrics[aug]['accuracy_gap']
            })
    
    # Add paper results
    for model_name, model_results in PAPER_RESULTS.items():
        for aug in AUGMENTATIONS_TO_EVAL:
            if aug in model_results:
                comparison_data.append({
                    'Model': model_name,
                    'Augmentation': aug,
                    'Within-Slide Acc': model_results[aug]['within_slide_accuracy'],
                    'Cross-Slide Acc': model_results[aug]['cross_slide_accuracy'],
                    'Accuracy Gap': model_results[aug]['accuracy_gap']
                })
    
    comparison_df = pd.DataFrame(comparison_data)
    
    # Plot comparison for each augmentation
    for aug in AUGMENTATIONS_TO_EVAL:
        if aug not in comparison_df['Augmentation'].values:
            continue
            
        aug_data = comparison_df[comparison_df['Augmentation'] == aug].sort_values('Accuracy Gap')
        
        fig, ax = plt.subplots(figsize=(10, 6))
        
        x = range(len(aug_data))
        width = 0.35
        
        ax.bar([i - width/2 for i in x], aug_data['Within-Slide Acc'], width,
               label='Within-Slide', alpha=0.8, color='#2ecc71')
        ax.bar([i + width/2 for i in x], aug_data['Cross-Slide Acc'], width,
               label='Cross-Slide', alpha=0.8, color='#3498db')
        
        # Highlight your model
        your_model_idx = aug_data[aug_data['Model'] == MODEL_ID.split('/')[-1]].index
        if len(your_model_idx) > 0:
            idx_pos = list(aug_data.index).index(your_model_idx[0])
            ax.axvspan(idx_pos - 0.5, idx_pos + 0.5, alpha=0.2, color='yellow', zorder=0)
        
        ax.set_xlabel('Model', fontsize=12)
        ax.set_ylabel('Accuracy', fontsize=12)
        ax.set_title(f'Model Comparison - {aug.replace("_", " ").title()}', 
                     fontsize=14, fontweight='bold')
        ax.set_xticks(x)
        ax.set_xticklabels(aug_data['Model'], rotation=45, ha='right')
        ax.legend()
        ax.grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Print ranking
        print(f"\n{aug.replace('_', ' ').title()} - Accuracy Gap Ranking (lower is better):")
        print("="*70)
        for i, (idx, row) in enumerate(aug_data.iterrows(), 1):
            marker = " ← YOUR MODEL" if row['Model'] == MODEL_ID.split('/')[-1] else ""
            print(f"{i}. {row['Model']:20s} | Gap: {row['Accuracy Gap']:.3f} | "
                  f"Within: {row['Within-Slide Acc']:.3f} | Cross: {row['Cross-Slide Acc']:.3f}{marker}")
        print("="*70)

else:
    print("⚠ No paper results available for comparison.")
    print("  Your model's results are displayed above.")

## Step 8: Embedding Space Analysis

Visualize and analyze the structure of the embedding space to understand feature collapse.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import numpy as np

# Select augmentation to visualize (prefer baseline if available)
viz_aug = 'baseline' if 'baseline' in all_results else list(all_results.keys())[0]
viz_data = all_results[viz_aug]

print(f"Visualizing embedding space for: {viz_aug}")
print(f"Total samples: {len(viz_data['embeddings'])}")

# Subsample for faster visualization if needed
max_samples = 2000
if len(viz_data['embeddings']) > max_samples:
    print(f"Subsampling to {max_samples} patches for visualization...")
    indices = np.random.choice(len(viz_data['embeddings']), max_samples, replace=False)
    viz_embeddings = viz_data['embeddings'][indices]
    viz_labels = viz_data['labels'][indices]
    viz_slide_ids = viz_data['slide_ids'][indices]
else:
    viz_embeddings = viz_data['embeddings']
    viz_labels = viz_data['labels']
    viz_slide_ids = viz_data['slide_ids']

# Apply PCA first for dimensionality reduction
print("Applying PCA...")
pca = PCA(n_components=50)
embeddings_pca = pca.fit_transform(viz_embeddings)
print(f"PCA explained variance: {pca.explained_variance_ratio_[:10].sum():.2%} (first 10 components)")

# Apply t-SNE
print("Applying t-SNE (this may take a few minutes)...")
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000)
embeddings_2d = tsne.fit_transform(embeddings_pca)

# Create visualizations
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Color by class label
ax1 = axes[0]
scatter1 = ax1.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], 
                       c=viz_labels, cmap='tab10', alpha=0.6, s=20)
ax1.set_title('Embedding Space - Colored by ISUP Grade', fontsize=14, fontweight='bold')
ax1.set_xlabel('t-SNE Dimension 1', fontsize=12)
ax1.set_ylabel('t-SNE Dimension 2', fontsize=12)
cbar1 = plt.colorbar(scatter1, ax=ax1)
cbar1.set_label('ISUP Grade', fontsize=10)
ax1.grid(alpha=0.3)

# Plot 2: Color by slide ID
ax2 = axes[1]
unique_slides = np.unique(viz_slide_ids)
slide_to_int = {slide: i for i, slide in enumerate(unique_slides)}
slide_colors = np.array([slide_to_int[s] for s in viz_slide_ids])
scatter2 = ax2.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1],
                       c=slide_colors, cmap='tab20', alpha=0.6, s=20)
ax2.set_title('Embedding Space - Colored by Slide ID', fontsize=14, fontweight='bold')
ax2.set_xlabel('t-SNE Dimension 1', fontsize=12)
ax2.set_ylabel('t-SNE Dimension 2', fontsize=12)
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("Interpretation Guide:")
print("="*70)
print("LEFT PLOT (colored by grade):")
print("  • Good: Clear separation between different grades")
print("  • Poor: Mixed colors throughout (grade confusion)")
print("\nRIGHT PLOT (colored by slide):")
print("  • Good: Mixed colors throughout (slide-independent features)")
print("  • Poor: Distinct clusters by color (slide-specific features)")
print("="*70)

## Step 9: Advanced Metrics and Analysis

Compute additional metrics including silhouette scores, k-NN analysis, and robustness measurements.

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.neighbors import NearestNeighbors

def compute_advanced_metrics(embeddings, labels, slide_ids, k=50):
    """Compute additional diagnostic metrics."""
    metrics = {}
    
    # Subsample for efficiency
    max_samples = 5000
    if len(embeddings) > max_samples:
        indices = np.random.choice(len(embeddings), max_samples, replace=False)
        embeddings = embeddings[indices]
        labels = labels[indices]
        slide_ids = slide_ids[indices]
    
    # Silhouette scores
    try:
        metrics['silhouette_class'] = silhouette_score(embeddings, labels)
    except:
        metrics['silhouette_class'] = 0.0
    
    try:
        metrics['silhouette_slide'] = silhouette_score(embeddings, slide_ids)
    except:
        metrics['silhouette_slide'] = 0.0
    
    # k-NN same-slide fraction
    nn = NearestNeighbors(n_neighbors=min(k, len(embeddings)-1))
    nn.fit(embeddings)
    
    same_slide_counts = []
    for i in range(len(embeddings)):
        distances, indices = nn.kneighbors([embeddings[i]])
        neighbor_slides = slide_ids[indices[0][1:]]  # Exclude self
        same_slide = (neighbor_slides == slide_ids[i]).sum()
        same_slide_counts.append(same_slide)
    
    metrics['knn_same_slide_fraction'] = np.mean(same_slide_counts) / (k - 1)
    
    # Slide ID prediction accuracy (how well can we predict slide from embeddings)
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import cross_val_score
    
    try:
        # Encode slide IDs as integers
        unique_slides = np.unique(slide_ids)
        slide_to_int = {slide: i for i, slide in enumerate(unique_slides)}
        slide_ints = np.array([slide_to_int[s] for s in slide_ids])
        
        clf = LogisticRegression(max_iter=1000, random_state=42)
        scores = cross_val_score(clf, embeddings, slide_ints, cv=5)
        metrics['slide_id_accuracy'] = scores.mean()
    except:
        metrics['slide_id_accuracy'] = 0.0
    
    return metrics

# Compute advanced metrics for each augmentation
print("Computing advanced metrics...\n")
advanced_metrics = {}

for aug, results in all_results.items():
    print(f"Processing {aug}...")
    adv_m = compute_advanced_metrics(
        results['embeddings'],
        results['labels'],
        results['slide_ids']
    )
    advanced_metrics[aug] = adv_m
    
    print(f"  Silhouette (class): {adv_m['silhouette_class']:7.3f} (higher = better class separation)")
    print(f"  Silhouette (slide): {adv_m['silhouette_slide']:7.3f} (lower = less slide clustering)")
    print(f"  k-NN same slide:    {adv_m['knn_same_slide_fraction']:7.3f} (lower = less slide bias)")
    print(f"  Slide ID predict:   {adv_m['slide_id_accuracy']:7.3f} (lower = less slide information)")
    print()

# Create comprehensive metrics table
comprehensive_data = []
for aug in all_results.keys():
    row = {
        'Augmentation': aug,
        'Within Acc': metrics[aug]['within_accuracy'],
        'Cross Acc': metrics[aug]['cross_accuracy'],
        'Gap': metrics[aug]['accuracy_gap'],
        'Sil-Class': advanced_metrics[aug]['silhouette_class'],
        'Sil-Slide': advanced_metrics[aug]['silhouette_slide'],
        'k-NN-Slide': advanced_metrics[aug]['knn_same_slide_fraction'],
        'Slide-ID': advanced_metrics[aug]['slide_id_accuracy']
    }
    comprehensive_data.append(row)

comp_df = pd.DataFrame(comprehensive_data)

print("\n" + "="*90)
print("Comprehensive Metrics Summary")
print("="*90)
print(comp_df.to_string(index=False, float_format='%.3f'))
print("="*90)

# Visualize advanced metrics
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

aug_names = [aug.replace('_', ' ').title() for aug in comp_df['Augmentation']]
x = range(len(comp_df))

# Silhouette scores
ax1 = axes[0, 0]
ax1.bar([i - 0.2 for i in x], comp_df['Sil-Class'], 0.4, label='Class', alpha=0.8, color='#2ecc71')
ax1.bar([i + 0.2 for i in x], comp_df['Sil-Slide'], 0.4, label='Slide', alpha=0.8, color='#e74c3c')
ax1.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax1.set_title('Silhouette Scores', fontweight='bold')
ax1.set_ylabel('Score')
ax1.set_xticks(x)
ax1.set_xticklabels(aug_names, rotation=45, ha='right')
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# k-NN same slide fraction
ax2 = axes[0, 1]
colors = ['#e74c3c' if v > 0.6 else '#f39c12' if v > 0.5 else '#2ecc71' 
          for v in comp_df['k-NN-Slide']]
ax2.bar(x, comp_df['k-NN-Slide'], alpha=0.8, color=colors)
ax2.axhline(y=0.5, color='red', linestyle='--', alpha=0.5, label='High bias (>0.50)')
ax2.set_title('k-NN Same-Slide Fraction', fontweight='bold')
ax2.set_ylabel('Fraction')
ax2.set_xticks(x)
ax2.set_xticklabels(aug_names, rotation=45, ha='right')
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

# Slide ID prediction accuracy
ax3 = axes[1, 0]
colors = ['#e74c3c' if v > 0.8 else '#f39c12' if v > 0.7 else '#2ecc71' 
          for v in comp_df['Slide-ID']]
ax3.bar(x, comp_df['Slide-ID'], alpha=0.8, color=colors)
ax3.axhline(y=0.8, color='red', linestyle='--', alpha=0.5, label='High leakage (>0.80)')
ax3.set_title('Slide ID Prediction Accuracy', fontweight='bold')
ax3.set_ylabel('Accuracy')
ax3.set_xticks(x)
ax3.set_xticklabels(aug_names, rotation=45, ha='right')
ax3.legend()
ax3.grid(axis='y', alpha=0.3)

# Accuracy gap (repeated for comparison)
ax4 = axes[1, 1]
colors = ['#e74c3c' if v > 0.2 else '#f39c12' if v > 0.1 else '#2ecc71' 
          for v in comp_df['Gap']]
ax4.bar(x, comp_df['Gap'], alpha=0.8, color=colors)
ax4.axhline(y=0.2, color='red', linestyle='--', alpha=0.5)
ax4.set_title('Accuracy Gap', fontweight='bold')
ax4.set_ylabel('Gap')
ax4.set_xticks(x)
ax4.set_xticklabels(aug_names, rotation=45, ha='right')
ax4.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*90)
print("Metric Interpretation:")
print("="*90)
print("Silhouette (class):  Higher is better - indicates good class separation")
print("Silhouette (slide):  Lower is better - indicates less slide-specific clustering")
print("k-NN same slide:     Lower is better - fewer neighbors from same slide")
print("Slide ID predict:    Lower is better - harder to identify slide from embeddings")
print("Accuracy gap:        Lower is better - less overfitting to slide identity")
print("="*90)

## Step 10: Summary and Recommendations

Final assessment of your model's performance on the PANDA-PLUS-Bench benchmark.

In [ ]:
def assess_model_performance(metrics_dict, advanced_metrics_dict):
    """Generate assessment and recommendations."""
    
    # Calculate average metrics
    avg_gap = np.mean([m['accuracy_gap'] for m in metrics_dict.values()])
    avg_cross_acc = np.mean([m['cross_accuracy'] for m in metrics_dict.values()])
    avg_knn_slide = np.mean([m['knn_same_slide_fraction'] for m in advanced_metrics_dict.values()])
    avg_slide_id = np.mean([m['slide_id_accuracy'] for m in advanced_metrics_dict.values()])
    
    # Scoring
    scores = {
        'Feature Collapse': 'Excellent' if avg_gap < 0.15 else 'Good' if avg_gap < 0.20 else 'Moderate' if avg_gap < 0.25 else 'Poor',
        'Cross-Slide Performance': 'Excellent' if avg_cross_acc > 0.55 else 'Good' if avg_cross_acc > 0.50 else 'Moderate' if avg_cross_acc > 0.45 else 'Poor',
        'Slide Independence': 'Excellent' if avg_knn_slide < 0.50 else 'Good' if avg_knn_slide < 0.55 else 'Moderate' if avg_knn_slide < 0.60 else 'Poor',
        'Information Leakage': 'Excellent' if avg_slide_id < 0.75 else 'Good' if avg_slide_id < 0.80 else 'Moderate' if avg_slide_id < 0.85 else 'Poor'
    }
    
    return scores, avg_gap, avg_cross_acc, avg_knn_slide, avg_slide_id

# Generate assessment
scores, avg_gap, avg_cross_acc, avg_knn_slide, avg_slide_id = assess_model_performance(
    metrics, advanced_metrics
)

print("\n" + "="*70)
print(f"PANDA-PLUS-Bench Assessment: {MODEL_ID}")
print("="*70 + "\n")

# Overall score
overall_ratings = list(scores.values())
if overall_ratings.count('Excellent') >= 3:
    overall = "🌟 EXCELLENT"
    color = '\033[92m'  # Green
elif overall_ratings.count('Poor') == 0:
    overall = "✓ GOOD"
    color = '\033[93m'  # Yellow
else:
    overall = "⚠ NEEDS IMPROVEMENT"
    color = '\033[91m'  # Red

print(f"Overall Rating: {color}{overall}\033[0m\n")

# Detailed scores
print("Detailed Assessment:")
print("-" * 70)
for metric, rating in scores.items():
    emoji = "🌟" if rating == "Excellent" else "✓" if rating == "Good" else "○" if rating == "Moderate" else "✗"
    print(f"{emoji} {metric:25s}: {rating}")
print()

# Key metrics
print("Key Metrics:")
print("-" * 70)
print(f"  Average Accuracy Gap:          {avg_gap:.3f}")
print(f"  Average Cross-Slide Accuracy:  {avg_cross_acc:.3f}")
print(f"  Average k-NN Same-Slide:       {avg_knn_slide:.3f}")
print(f"  Average Slide ID Prediction:   {avg_slide_id:.3f}")
print()

# Recommendations
print("Recommendations:")
print("-" * 70)

if avg_gap > 0.20:
    print("⚠ HIGH FEATURE COLLAPSE DETECTED")
    print("  → Consider additional regularization during training")
    print("  → Implement slide-level data augmentation")
    print("  → Use contrastive learning with slide-aware negative sampling")
    
if avg_cross_acc < 0.50:
    print("⚠ LOW CROSS-SLIDE GENERALIZATION")
    print("  → Train on more diverse datasets")
    print("  → Implement domain adaptation techniques")
    print("  → Consider stain normalization preprocessing")
    
if avg_knn_slide > 0.60:
    print("⚠ HIGH SLIDE-SPECIFIC CLUSTERING")
    print("  → Embeddings are capturing slide-specific artifacts")
    print("  → Review preprocessing pipeline for scanner/stain bias")
    print("  → Consider adversarial training to remove slide information")
    
if avg_slide_id > 0.85:
    print("⚠ SEVERE INFORMATION LEAKAGE")
    print("  → Model is encoding slide identity in embeddings")
    print("  → Critical issue for downstream clinical applications")
    print("  → Requires model retraining with bias mitigation")

if avg_gap < 0.15 and avg_cross_acc > 0.50:
    print("✓ EXCELLENT PERFORMANCE")
    print("  → Model shows strong generalization")
    print("  → Low feature collapse indicates robust representations")
    print("  → Suitable for multi-site clinical deployment")

print("\n" + "="*70)
print("Next Steps:")
print("="*70)
print("1. Test on additional augmentation conditions for robustness")
print("2. Evaluate on external validation cohorts")
print("3. Compare with other foundation models using the rankings above")
print("4. Consider fine-tuning on downstream tasks with appropriate safeguards")
print("5. Share your results with the community!")
print("\n" + "="*70)

# Create a final summary visualization
fig, ax = plt.subplots(figsize=(10, 6))

categories = list(scores.keys())
ratings_numeric = {
    'Excellent': 4,
    'Good': 3,
    'Moderate': 2,
    'Poor': 1
}
values = [ratings_numeric[scores[cat]] for cat in categories]
colors_map = {4: '#2ecc71', 3: '#3498db', 2: '#f39c12', 1: '#e74c3c'}
bar_colors = [colors_map[v] for v in values]

y_pos = range(len(categories))
bars = ax.barh(y_pos, values, color=bar_colors, alpha=0.8)

ax.set_yticks(y_pos)
ax.set_yticklabels(categories)
ax.set_xlabel('Rating Score', fontsize=12)
ax.set_title(f'PANDA-PLUS-Bench Performance Summary\n{MODEL_ID}', 
             fontsize=14, fontweight='bold')
ax.set_xlim(0, 4.5)
ax.set_xticks([1, 2, 3, 4])
ax.set_xticklabels(['Poor', 'Moderate', 'Good', 'Excellent'])
ax.grid(axis='x', alpha=0.3)

# Add value labels
for i, (bar, val, rating) in enumerate(zip(bars, values, scores.values())):
    ax.text(val + 0.1, i, rating, va='center', fontweight='bold')

plt.tight_layout()
plt.show()